In [184]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
#import inflect #numeric/cardinal

In [185]:
dirPath = r"C:\Study\Projects\NLP\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"
qaPath = r"C:\Study\Projects\NLP\data" 
path = dirPath + "\\" + reviewPath

In [186]:
dataFile = gzip.open(path, 'r')

In [187]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [188]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [189]:
len(productReviews)

134476

In [190]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [191]:
productsReviewData.shape
#productsReviewData.columns
productsReviewData.head(1)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A4IL0CLL27Q33,104800001X,D. Brennan,"[0, 1]","I hate it when my shirt collars, not otherwise secured in place by buttons, end up in weird places throughout the day. I purchased some steel collar stays to use with these magnets but they were only vaguely magnetic. I ended up using 2 of these magnets - one in the collar with the stay and the other inside my shirt, to lock my collar in place. They work flawlessly. They are the perfect size, and there are plenty of magnets in case you forget to remove them at the end of the day.",5.0,Perfect for collar stay management,1390953600,"01 29, 2014"


In [192]:
reduceData = productsReviewData.loc[:, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]
print(reduceData.shape)
#reduceData = reduceData.loc[reduceData.overall != 3.0, :] #Ignore rating at 3 
#reduceData.index = range(0, len(reduceData), 1)  #very important if we drop or excluse certain items
#print(reduceData.shape)
#reduceData

(134476, 5)


In [245]:
#rating = lambda x: 1 if x > 3.0 else 0 # %84 accuracy > (14, 121, 63, 802) and BlobText (33, 102, 34, 831)
rating = lambda x: 1 if x > 2.0 else 0 #%  94% accuracy > (0, 46, 0, 954) and BlobText 14, 28, 57, 901

In [246]:
reduceData['rating'] = reduceData['overall'].apply(rating)
#unfavorableReviews = (reduceData.loc[ reduceData['rating'] == 0, 
#                                         ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall'))
#unfavorableReviews

In [247]:
reviewTextProcessing = pd.DataFrame(reduceData.reviewText.copy())

In [248]:
reviewTextProcessing['rating'] = reduceData['rating']
reviewTextProcessing['overall'] = reduceData['overall']

In [249]:
productsReviewData.dtypes
len(productsReviewData)
# reviewText, rating, overall
reviewTextProcessing.shape

(134476, 3)

In [250]:
#reviewTextProcessing.loc[reviewTextProcessing['rating']==0, ['rating', 'overall', 'Processed', 'SentimentModel']].head(2)

In [251]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

#print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)



In [200]:
sample1 = "s***"
print(re.sub("\*", "#", sample1))

sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)
expand_contractions(sample1)

s###


'shit'

In [201]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
        eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        #reconstruct sentence
        review[count] = " ".join(eachSent)
    #a clobbed together without pun
    wholeText = [" ".join(review) ]
    #reconstruct paragraph of sentences

    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

sample = "This is a test. It is only a test!"
paragraphText, wholeText = singleReviewCleanUp(sample)
wholeText, paragraphText
#paragraphText

(['this is a test it is only a test'], ['this is a test', 'it is only a test'])

In [252]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [253]:
%%time
reviewTextProcessing['Processed'] = reviewTextProcessing.reviewText.apply(cleanUpTextParagraph)


Wall time: 3min 40s


In [254]:
reviewTextProcessing.Processed.head(5)

0                          [i hate it when my shirt collars not otherwise secured in place by buttons end up in weird places throughout the day, i purchased some steel collar stays to use with these magnets but they were only vaguely magnetic, i ended up using of these magnets one in the collar with the stay and the other inside my shirt to lock my collar in place, they work flawlessly, they are the perfect size and there are plenty of magnets in case you forget to remove them at the end of the day]
1                                                                                                                                                                                                                                                                                                                                                            [these little magnets are really powerful for there size, i am using them to make secret compartments in custom made boxes, each one hols about of

In [205]:
#sample = " I don't know what to do. I need to test through tough. Do you have any suggestions"
#result = cleanUpTextParagraph(sample)
#result

In [206]:
from textblob import TextBlob

# need to do a data check first
def getPolaritySubjectivity(data):
    #print(data)
    paragraphText = data
    sumPolarity = []
    sumSubjectivity = []
    for each in paragraphText:
        sumPolarity.append(TextBlob(each).sentiment.polarity)
        sumSubjectivity.append(TextBlob(each).sentiment.subjectivity)
    #print("sumPol", sumPolarity)
    return sumPolarity, sumSubjectivity
    #return sum(sumPolarity), sum(sumSubjectivity)

    #print(each, TextBlob(each).sentiment.polarity, TextBlob(each).sentiment.subjectivity)
#(sample2, TextBlob(sample2).sentiment.polarity, TextBlob(sample2).sentiment.subjectivity)


In [207]:
#Return the sume of polarity in a paragraph sentence when give a text in a list form of sentence

getPolarityFromText = lambda text: getPolaritySubjectivity(text)[0]

In [208]:
#for each in sampleText:
#    sample = cleanUpTextParagraph(each)
#    print(sample, getPolarityFromText(sample))

In [209]:
#print(stopWords)

In [210]:
#sentiment = lambda text: TextBlob(text).sentiment.polarity

In [211]:
adjustSentimentModel = lambda x: [sum(x), 0.0] if sum(x) < 0 else [sum(x), 1.0]

In [290]:
%%time
reviewTextProcessing['Sentiment']= reviewTextProcessing.Processed.apply(getPolarityFromText)

Wall time: 7min 17s


In [291]:
%%time
reviewTextProcessing['SentimentModel']= reviewTextProcessing.Sentiment.apply(adjustSentimentModel)

Wall time: 255 ms


In [214]:
#reviewTextProcessing.loc[unfavorableReviews, ['Processed', 'Sentiment', 'SentimentModel']].head(2)
#reviewTextProcessing.dtypes

In [292]:
reviewTextProcessing['CleanedJoin'] = reviewTextProcessing['Processed'].apply(', '.join)


In [257]:
reviewTextProcessing.shape
#reviewTextProcessing.columns

(134476, 5)

In [258]:
#from gensim import corpora
#dictionary = corpora.Dictionary(T)
#print(dictionary.token2id)

In [293]:
reviewTextProcessing['CleanedJoin'].head(2)

0    i hate it when my shirt collars not otherwise secured in place by buttons end up in weird places throughout the day, i purchased some steel collar stays to use with these magnets but they were only vaguely magnetic, i ended up using of these magnets one in the collar with the stay and the other inside my shirt to lock my collar in place, they work flawlessly, they are the perfect size and there are plenty of magnets in case you forget to remove them at the end of the day
1                                                                                                                                                                                                                                                                                                                                      these little magnets are really powerful for there size, i am using them to make secret compartments in custom made boxes, each one hols about of a pound
Name: CleanedJoin, dtype: object

In [260]:

import itertools

largeText = reviewTextProcessing.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

134476

In [294]:
sentiment = reviewTextProcessing['SentimentModel'].to_list()

In [221]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

num_words = 10000


In [262]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [263]:

size = int(len(largeText)*.75)
x_train_text = largeText[:size]
x_test_text = largeText[size:]

y_train =  reviewTextProcessing['rating'][:size]
y_test = reviewTextProcessing['rating'][size:]
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)


In [264]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(100857, 33619, 100857, 33619)

In [225]:
#x_train_tokens[1], x_train_text[1]

In [265]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [266]:
np.mean(num_tokens), np.max(num_tokens)


(110.80553407299443, 4838)

In [ ]:
## np.std(num_tokens)

In [267]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


376

In [268]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.958706386269669

In [269]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape
x_test_pad.shape


(33619, 376)

In [231]:
index = len(x_train_pad)
index

100857

In [232]:
for i in range(0, 1):
    print(x_train_pad[i])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [233]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [234]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [52]:
x_train_text[1]


'the product does exactly as it should and is quite affordable i did not realized it was double screened until it arrived so it was even better than i had expected as an added bonus one of the screens carries a small hint of the smell of an old grape candy i used to buy so for reminiscent s sake i can not stop putting the pop filter next to my nose and smelling it after recording, dif you needed a pop filter this will work just as well as the expensive ones and it may even come with a pleasing aroma like mine did buy this product, '

In [53]:
tokens_to_string(x_train_tokens[1])


'the product does exactly as it should and is quite affordable i did not realized it was double screened until it arrived so it was even better than i had expected as an added bonus one of the screens carries a small hint of the smell of an old grape candy i used to buy so for reminiscent s sake i can not stop putting the pop filter next to my nose and it after recording dif you needed a pop filter this will work just as well as the expensive ones and it may even come with a pleasing like mine did buy this product'

In [148]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [270]:
model = Sequential()
#embedding_size = 8 > 0.8694
embedding_size = 8*3
optimizer = Adam(lr=1e-3)


In [271]:
model.add(Embedding(input_dim=num_words,output_dim=embedding_size, input_length=max_tokens, name='layer_embedding'))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [277]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [278]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer2_embedding (Embedding) (None, 376, 24)           240000    
_________________________________________________________________
gru_9 (GRU)                  (None, 376, 16)           1968      
_________________________________________________________________
gru_10 (GRU)                 (None, 376, 8)            600       
_________________________________________________________________
gru_11 (GRU)                 (None, 4)                 156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 242,729
Trainable params: 242,729
Non-trainable params: 0
_________________________________________________________________


In [279]:
# %%time
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=5, batch_size=64)

model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=64)

Train on 95814 samples, validate on 5043 samples
Epoch 1/3
95814/95814 [==============================] - 747s 8ms/sample - loss: 0.2247 - acc: 0.9282 - val_loss: 0.1851 - val_acc: 0.9334
Epoch 2/3
95814/95814 [==============================] - 734s 8ms/sample - loss: 0.1559 - acc: 0.9424 - val_loss: 0.1717 - val_acc: 0.9377
Epoch 3/3
95814/95814 [==============================] - 758s 8ms/sample - loss: 0.1313 - acc: 0.9513 - val_loss: 0.1747 - val_acc: 0.9328


In [280]:
%%time
result = model.evaluate(x_test_pad, y_test)



33619/33619 [==============================] - 106s 3ms/sample - loss: 0.1655 - acc: 0.9394
Wall time: 1min 46s


In [281]:
print("Accuracy: {0:.2%}".format(result[1]))


Accuracy: 93.94%


In [282]:
%%time
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

Wall time: 3.34 s


In [295]:
y_pred

array([0.99408984, 0.9936342 , 0.9957746 , 0.99258053, 0.9948828 ,
       0.9936437 , 0.9960873 , 0.99284875, 0.9952526 , 0.99552345,
       0.11372644, 0.9964645 , 0.98891956, 0.9962717 , 0.898716  ,
       0.9728962 , 0.9957236 , 0.80997586, 0.99564236, 0.9959892 ,
       0.98257905, 0.3086452 , 0.9891758 , 0.75045896, 0.99194795,
       0.98051167, 0.9957129 , 0.9612137 , 0.99216163, 0.98338556,
       0.99241745, 0.99624395, 0.99366236, 0.99508166, 0.99245584,
       0.99513936, 0.9939476 , 0.31506366, 0.995342  , 0.92726564,
       0.9898269 , 0.99468434, 0.9953711 , 0.9949105 , 0.9503124 ,
       0.9897896 , 0.9937831 , 0.98829925, 0.41269282, 0.98115206,
       0.99628985, 0.9960763 , 0.98378086, 0.9809071 , 0.99067473,
       0.9660648 , 0.99605197, 0.9942354 , 0.8798444 , 0.9962816 ,
       0.9963231 , 0.9945872 , 0.99569094, 0.9782295 , 0.9950129 ,
       0.99038684, 0.9958422 , 0.9870006 , 0.9956392 , 0.99580896,
       0.9931312 , 0.9958078 , 0.99489236, 0.99514717, 0.99649

In [296]:
list(y_pred)
result_location = np.where(y_pred.T)[0]  + index
RNNresult = reviewTextProcessing.loc[list(result_location),  ['reviewText', 'overall', 'rating', 'Sentiment' , 'SentimentModel']]
RNNresult.head(2)

,reviewText,overall,rating,Sentiment,SentimentModel
100857,"Hunter 90434 16-Inch Portable Stand Fan, Oil Rubbed BronzeThis review is about the Hunter 90434 16-Inch Portable Stand Fan, Oil Rubbed Bronze Fan. It's a very nice oscillating 3 speed adjustable height fan. You assemble it your self. When attempting to assemble the fan I found out I had a striped screw. So I contacted Hunter Fan company and they shipped me a new fan. Which assembled with no problems. I set my fan in my sliding glass windows when it cools off at night,it does a very nice job ...",5.0,1,"[0.0, 0.78, 0.0, 0.0, 0.13636363636363635, 0.0, 0.89, 0.0]","[1.8063636363636364, 1.0]"
100858,"Needed these because I was refinishing a large dining table. Worked as expected, good tack, needed only about 1/4 of a single cloth, using it for several stages. Seems like a quality product made in the USA - not Chinese c**p. Each cloth individually wrapped, so he package should last a very long time.",5.0,1,"[0.21428571428571427, 0.10571428571428572, 0.0, -0.021666666666666667]","[0.29833333333333334, 1.0]"


In [297]:
pd.DataFrame(y_pred).describe()

,0
count,1000.000000
mean,0.925989
std,0.182131
min,0.044473
25%,0.975933
50%,0.992860
75%,0.995464
max,0.996543


In [298]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])


In [299]:
RNNresult['RNN_Sentiment'] = cls_pred

In [300]:
sumSentiment = lambda x: x[1]

In [302]:
RNNresult.columns = ['TextBlobAnalysis' if x=='Sentiment' else x for x in RNNresult.columns]
RNNresult.columns = ['TextBlob_Sentiment' if x=='SentimentModel' else x for x in RNNresult.columns]
RNNresult['TextBlob_Sentiment'] = RNNresult['TextBlob_Sentiment'].apply(sumSentiment)
RNNresult.head(3)
RNNresult.shape

(1000, 6)

In [175]:
rating = np.array(RNNresult.rating)
RNN = np.array(RNNresult.RNN_Sentiment)
TextBlob = np.array(RNNresult.TextBlob_Sentiment)

In [303]:
#RNNresult.loc[RNNresult.RNN_Sentiment < 1, :]
RNNresult.loc[RNNresult.overall >= 3.0 , :].to_csv("Rating3.0above_RNN.csv")
RNNresult.loc[RNNresult.overall < 3.0 , :].to_csv("RatingBelow3.0_RNN.csv")

In [304]:
RNNresult.RNN_Sentiment = RNNresult.RNN_Sentiment.astype(int)
RNNresult.TextBlob_Sentiment = RNNresult.TextBlob_Sentiment.astype(int)

In [177]:
comparison = TextBlob + rating + RNN
details = np.array([True if (x == 3.0 or x == 0) else False for x in comparison])
mismatch = np.where(details == False)
#confusionMaxtrix = RNNresult.loc[list(index + (mismatch[0])), :]
#confusionMaxtrix.loc[RNNresult.overall > 2.0 ,  :]
#confusionMaxtrix.RNN_Sentiment = confusionMaxtrix.RNN_Sentiment.astype(int)
#confusionMaxtrix.TextBlob_Sentiment = confusionMaxtrix.TextBlob_Sentiment.astype(int)

In [305]:
from sklearn.metrics import confusion_matrix

y_true = RNNresult.rating.to_list()
rnn_pred = RNNresult.RNN_Sentiment.to_list()
textBlob_pred = RNNresult.TextBlob_Sentiment.to_list()



In [306]:
rrn_compare = confusion_matrix(y_true, rnn_pred)
textBlob_compare =  confusion_matrix(y_true, textBlob_pred)

In [307]:
tn, fp, fn, tp = rrn_compare.ravel()
tn, fp, fn, tp   

(35, 26, 25, 914)

In [308]:
#result tn, fp, fn, tp = textBlob_compare.ravel()
#textBlobResult = np.array(tn, fp, fn, tp)
result = textBlob_compare.ravel()
tn, fp, fn, tp = result
tn, fp, fn, tp

(21, 40, 93, 846)

In [79]:
RNNresult.loc[[7736, 7750, 7911], :].to_csv("NegativeReviews_RNN.csv")

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

array = [[13,1,1,0,2,0],
     [3,9,6,0,1,0],
     [0,0,16,2,0,0],
     [0,0,0,13,0,0],
     [0,0,0,0,15,0],
     [0,0,1,0,0,15]]        

classNames = ['Negative','Positive']
plt.title('Confusion Maxtrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
#tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
df_cm = pd.DataFrame(result, range(2),
                  range(2))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.40)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 16})# 
df_cm

In [ ]:
cm = result
#plt.clf()
#plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Wistia)
classNames = ['Negative','Positive']
plt.title('Confusion Maxtrix')
plt.ylabel('True label')
plt.xlabel('Predicted label')
#tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.show()

In [ ]:
cls_true = np.array(y_test[0:1000])

#y_test

In [ ]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [ ]:
len(incorrect)


In [ ]:
reduceData.loc[7695+40, ['reviewText', 'overall', 'rating']]
reviewTextProcessing.loc[7695+40, ['CleanedJoin', 'overall', 'rating', 'SentimentModel']]

In [ ]:
idx = incorrect[1]
print(type(idx))
for each in incorrect[:10]:
    idx = each
    text = x_test_text[idx]
    print(idx, text, cls_pred[idx], cls_true[idx], " BlobText ",
         TextBlob(text).sentiment.polarity)


In [ ]:
reviewTextProcessing.loc[list(incorrect + 7695), ['CleanedJoin', 'overall', 'rating', 'SentimentModel']]

In [ ]:

top_dict = {}
#for c in data_dtm.columns:
#    top = data_dtm[c].sort_values(ascending=False).head(30)
#    top_dict[c]= list(zip(top.index, top.values))

top_dict['cable'] = 3
top_dict['not happy'] = 4
top_dict['disappoint'] = 7

regeneration = ""

for key, value in top_dict.items():
    space = ((key) + " ")
    details = (space) * value
    regeneration = regeneration + details
    
review = " ".join(top_dict.keys())
top_dict.values()
regeneration



In [ ]:
from wordcloud import WordCloud

#stopwords=stopWords,
wc = WordCloud(background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

In [ ]:
import string
string.punctuation

In [ ]:
#handling money
#>>> a = re.findall(r"\$\d+.\d+", text2)

In [ ]:
#numeric and fraction handling
#domain specific if applies to tool/measurement?
pattern = re.compile(r'^\d+/\d+$')

def evaluateFraction(data, pattern=pattern):
    try:
        re.match(pattern, data)
        return eval(data)  
    except:
        print(f"Not fraction related input {data}")
        return False      

In [77]:
pd.options.display.max_colwidth = 500

#productsReviewData.cleanedText.head(2)

In [ ]:
sampleData = productsReviewData

In [ ]:
reviewTextProcessing.loc[[100867, 101061, 101054, 101047, 101163
                         ], ['reviewText', 'overall', 'rating', 'SentimentModel']]